In [1]:
import threading
import time
import math

import pandas as pd
import numpy as np
from IPython.display import Image
np.set_printoptions(precision = 3)

In [2]:
data = pd.read_csv("jester-data-1.csv", header=None)

math.log10(0.00001)  
-5.0

In [3]:
#user_ratings = data.values
user_ratings = data.values[0:100]

In [4]:
n_features = 7
latent_user_preferences = np.random.random((user_ratings.shape[0], n_features))
latent_item_features = np.random.random((user_ratings.shape[1],n_features))

In [5]:
print('user_ratings.shape:           ',user_ratings.shape)
print('latent_user_preferences.shape:',latent_user_preferences.shape)
print('latent_item_features.shape:   ',latent_item_features.shape)

user_ratings.shape:            (100, 101)
latent_user_preferences.shape: (100, 7)
latent_item_features.shape:    (101, 7)


In [6]:
user_ratings_vali = np.zeros(user_ratings.shape) + 99.0
user_ratings_copy = user_ratings.copy() 
count = 0
for x in range(0,user_ratings.shape[0]):
    for y in range(0,user_ratings.shape[1]):
        if (user_ratings_copy[x,y]!=99.0):
            if (np.random.random() > 0.9):
                count += 1
                temp = user_ratings_copy[x,y]
                user_ratings_copy[x,y] = user_ratings_vali[x,y]
                user_ratings_vali[x,y] = temp
                #print(temp)
print(user_ratings_vali,'\n', count,'\n', count/(user_ratings.shape[0]*user_ratings.shape[1]))

[[99.   99.   99.   ... 99.   99.   99.  ]
 [99.   99.   -0.29 ... 99.   99.   99.  ]
 [99.   99.   99.   ... 99.   99.   99.  ]
 ...
 [99.   99.   99.   ... 99.   99.   99.  ]
 [99.   99.   99.   ... 99.   99.   99.  ]
 [99.   99.   99.   ... 99.   99.   99.  ]] 
 740 
 0.07326732673267326


In [7]:
def predict_rating(user_id,item_id):
    """ Predict a rating given a user_id and an item_id.
    """
    user_preference = latent_user_preferences[user_id]
    item_preference = latent_item_features[item_id]
    return user_preference.dot(item_preference)

def train(user_id, item_id, rating,alpha = 0.0001):
    
    #print (item_id)
    prediction_rating = predict_rating(user_id, item_id)
    err =  ( prediction_rating- rating );
    #print (err)
    user_pref_values = latent_user_preferences[user_id][:]
    latent_user_preferences[user_id] -= alpha * err * latent_item_features[item_id]
    latent_item_features[item_id] -= alpha * err * user_pref_values
    return err
    


def sgd(iterations = 100000):
    """ Iterate over all users and all items and train for 
        a certain number of iterations
    """
    err_mse_log = 0
    for iteration in range(0,iterations):
        error = []
        for user_id in range(0,latent_user_preferences.shape[0]):
            for item_id in range(0,latent_item_features.shape[0]):
                rating = user_ratings_copy[user_id][item_id]
                if(rating != 99.0):
                    err = train(user_id,item_id,rating)
                    error.append(err)
                    #print(rating)
        mse = (np.array(error) ** 2).mean()
        #print(iteration)
        if(iteration%1000 == 0 ):
            err_mse = mse - err_mse_log
            err_mse_log = mse
            print (iteration, ':=', mse, err_mse)
                    
                    

In [8]:
threads = []
for i in range(2):
    t = threading.Thread(target=sgd)
    threads.append(t)
    t.start()

0 := 100.11951051719248 100.11951051719248
0 := 100.65949903301293 100.65949903301293
1000 := 11.578249728219305 -89.08124930479363
1000 := 11.570344869334486 -88.54916564785799
2000 := 11.48636355266864 -0.09188617555066436
2000 := 11.482614639842353 -0.0877302294921325


kill all